In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    import os
    import sys
    os.chdir('/content/drive/MyDrive/DL_Project_GD/')
    sys.path.append('/content/drive/MyDrive/DL_Project_GD/code/')

    #%% [code]
    ! pip install -qqq "torchvision" "torch>=1.6, <1.9" "torchmetrics>=0.3" "ipython[notebook]" "pytorch-lightning>=1.3" "torchtext"
    ! pip install -qqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.8-cp37-cp37m-linux_x86_64.whl

    ## Install dependencies
    ! pip install -qqq wget
    ! apt-get -qqq install sox libsndfile1 ffmpeg
    ! pip install -qqq unidecode
    ! pip install -qqq matplotlib>=3.3.2

    ## Install NeMo
    BRANCH = 'r1.6.1'
    ! python -m pip install -qqq git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

    ! mkdir configs
    ! wget -P configs/ https://raw.githubusercontent.com/NVIDIA/NeMo/$BRANCH/examples/asr/conf/config.yaml

    datapath = './SyncData/wav/'
    rootpath = './'
    print("Running with Colab setup.")
    tpu_cores = 8
    gpu_cores = None
except(ModuleNotFoundError):
    print('Initialising non-colab set-up...')
    tpu_cores=None
    gpu_cores=None
    datapath = '../dataset/processed/'
    rootpath = '../'

from VGGmini import VGGmini
from dataloader import VoxDataloader
import runner
import os
print("cwd: {}".format(os.getcwd()))
print("datapath: {}".format(datapath))
print('done.')

In [ ]:
config = {
            # learning params
            'lr': 1e-3,
            'batch_size': 64,
            'early_stopping': True,
            'optimizer': 'Adam',
            'max_epochs': 20,
            'early_stopping': True,
            'patience': 3, # tolerance for early stopping
            'momentum': 0.,
            'lr_decay': 0.,

            # model
            'fftmethod': 'librosa.stft', # choose librosa.stft, signal.stft, or librosa.mfcc
            'pooling' : 'average', # 'average' or 'max'. will not have effect if not using VGGpool

            #, regularization
            'dropout': 0.0,
            'L2': 0.0,
            'batch_norm': True,
            'gradient_clipping': 0.0,
}

In [ ]:
for L2 in [0, 0.2, 0.4, 0.6, 0.8]:
  config['L2'] = L2
  runner.run(config, datapath, VoxDataloader, VGGmini, './VGGmini/', tpu_cores=tpu_cores, gpu_cores=gpu_cores, phase_map_file='phase_map_med.csv')
